## Running the clean function

In [1]:
def clean_data(df):
    cols = [] # Making all columns lower case
    for column in df.columns:
        cols.append(column.lower())
    cols
    df.columns = cols 
    df=df.drop(['id', 'age', 'growth'], axis=1) # droping irrelevant columns
    df=df[df['volleys'].isna()==False] # filling all the nulls
    df['composure'].fillna(df['composure'].median(), inplace=True)
    df_to_clean = ['gk', 'rb', 'rcb', 'cb', 'lcb', 'lb', 'rwb', 'rdm', 'cdm', 'ldm', 'lwb', 'rm', 'rcm', 'cm', 'lcm', 'lm', 'ram', 'cam', 'lam', 'rw', 'rf', 'cf', 'lf', 'lw', 'rs', 'st', 'ls'] #ating all the categoricals columns that I want to convert into numerical that is required for the prediction model 
    for col in df_to_clean:
        df[col] = df[col].str.split('+').str[0]
        df[col] = df[col].str.split('-').str[0]
    df[df_to_clean]= df[df_to_clean].apply(pd.to_numeric)
    df=pd.concat([df, df[df_to_clean]], axis=1)
    df=df.select_dtypes(include=np.number) # attributing all the cleaned data and only the numerical columns to a variable
    
    return df

    

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score 
import pickle

In [3]:
df=pd.read_csv('fifa21_validate.csv')

In [4]:
df=clean_data(df)

In [5]:
display(df.columns)
display(df.shape)

Index(['attacking', 'crossing', 'finishing', 'heading accuracy',
       'short passing', 'volleys', 'skill', 'dribbling', 'curve',
       'fk accuracy',
       ...
       'cam', 'lam', 'rw', 'rf', 'cf', 'lf', 'lw', 'rs', 'st', 'ls'],
      dtype='object', length=104)

(1996, 104)

## It has the same columns as my trained model. 
## Assigning new X-y Split

In [6]:
y=df['ova']
X = df.drop(['ova'], axis=1)

## Normalizing the new dataframe with same method

In [7]:
transformer=pickle.load(open('minmaxtransformer.p', 'rb'))


In [8]:
transformer

MinMaxScaler()

In [9]:
x_normalized = transformer.transform(X)
df=pd.DataFrame(x_normalized, columns=X.columns)

## Loading the fitted model and applying to the new X/y values

In [10]:
lm=pickle.load(open('model.p', 'rb'))

In [11]:
lm

LinearRegression()

In [12]:
predictions_test = lm.predict(X)
r2_score(y, predictions_test)

0.8879338065185125

## RMSE score for the validation data

In [13]:
rmse = np.sqrt(mean_squared_error(y,predictions_test))  
rmse

2.2630833675580213